# 1 : Librairies Python

In [1]:
# Librairies et options :
# Divers :
import pandas as pd
import datetime
import requests
import json
import numpy as np
import tqdm

# Importer les codes depuis un fichier .env :
from dotenv import load_dotenv
import os
load_dotenv()

# Création et lecture des BDD postgresl :
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy.types import Integer, BigInteger, JSON

# 2 : Clés API et BDD :

In [2]:
# Informations API : https://weatherlink.github.io/v2-api/
# Clés API :
APIKey = os.getenv("APIKey")
APISecret = os.getenv("APISecret")
stationID = os.getenv("stationID")

In [3]:
# Paramètres de connexion à la base de données PostgreSQL en local :
host = os.getenv("host")
database = os.getenv("database")
user = os.getenv("user")
password = os.getenv("password")
nomTable = os.getenv("nomTable")

# 3 : Récupération du dernier TS de la table ou date du début de la sonde :

In [4]:
def startEndDateAPI () :
    
    # Enddate : aujourd'hui à minuit en TS :
    today = datetime.date.today()
    todayMidnight = datetime.datetime.combine(today, datetime.time.min)
    endDate = int(todayMidnight.timestamp())
    
    # Startdate : 1er jour de la sonde ou dernier TS enregistré dans la BDD :
    try : # Présence d'une TS dans la table :
        # Connexion à la base de données
        conn = psycopg2.connect(dbname = database, user = user, password = password , host = host)
        cur = conn.cursor()

        # Exécution d'une requête SQL et récupération de la TS : 
        cur.execute(f"SELECT ts FROM {nomTable} ORDER BY ts DESC LIMIT 1")
        data = cur.fetchall()
        startDate = pd.DataFrame(data, columns=[desc[0] for desc in cur.description]).values[0][0]
        ifExists = 'append' # informations pour la BDD

        # Fermeture du curseur et de la connexion
        cur.close()
        conn.close()
        
    except : # Pas de TS dans la table :
        # Date du début de la station en TS :
        startStation = datetime.datetime(2021, 9, 29, 0, 0)
        startDate = int(startStation.timestamp())
        ifExists = 'replace' # informations pour la BDD
        
    
    return startDate, endDate, ifExists

# 4 : Ouverture de l'API

In [5]:
# DataFrame historiques :
dfAjout = pd.DataFrame()

# Start et End date :
startDate = startEndDateAPI()[0]
endDate = startEndDateAPI()[1]

# Nb de jours à récupérer :
nbJours = int((endDate - startDate) / 86400)

for i in tqdm.tqdm(range(nbJours)):
    startTime = startDate + i * 86400
    endTime = startTime + 86400
    
    # Lien de la request : 
    link = 'https://api.weatherlink.com/v2/historic/{}?api-key={}&start-timestamp={}&end-timestamp={}'.format(stationID, APIKey, startTime, endTime)
    headers = {'X-Api-Secret' : APISecret}

    # Requête :
    r = requests.get(link, headers=headers)

    # Si la requête a réussi :
    if r.status_code == 200:
        # Lecture de la request en json :
        data = r.json()

        # Transformation en DF : 
        dfJour = pd.DataFrame(data)
        dfJour = dfJour[['station_id','sensors']]

        # Récupération des valeurs se trouvant dans sensors :
        dfSensors = pd.json_normalize(data['sensors'][0]['data'])
    
        # Récupération des json sur une colonne :
        dfJour = pd.DataFrame({'station_id': data['station_id'], 'infos_json' : data['sensors'][0]['data']})
    
        # Convertir les objets JSON en chaînes de caractères JSON :
        dfJour['infos_json'] = dfJour['infos_json'].apply(json.dumps)

        # Concat des données :
        dfJour = pd.concat([dfJour, dfSensors], axis=1)
        
        # Concaténation des données :
        dfAjout = pd.concat([dfAjout, dfJour], ignore_index=True)
    else:
        print("La requête {} a échoué avec le code d'erreur {}".format(link, r.status_code))

0it [00:00, ?it/s]


In [6]:
# Data Frame Finale :
dfAjout.tail()

""


# 4 : Transfert sur PostgreSQL

In [7]:
# Création de la chaîne de connexion PostgreSQL :
connStr = f"postgresql://{user}:{password}@{host}/{database}"

# Création de la connexion à la base de données PostgreSQL :
engine = create_engine(connStr)

# Définir les types de données pour chaque colonne :
dtype = {'station_id': Integer(),
         'ts': BigInteger(),
         'infos_json': JSON}

# Insérer le DataFrame dans la base de données PostgreSQL :
dfAjout.to_sql(nomTable, engine, if_exists = startEndDateAPI()[2] , index=False, dtype=dtype)

# Fermeture de la connexion :
engine.dispose()

# 5 : Ouverture de la BDD

In [8]:
# Connexion à la base de données
conn = psycopg2.connect(dbname = database, user = user, password = password , host = host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour sélectionner les données de ma_table
cur.execute(f"SELECT * FROM {nomTable} LIMIT 5")

# Récupération des données dans une liste de tuples
data = cur.fetchall()

# Création d'un DataFrame à partir des données
df = pd.DataFrame(data, columns=[desc[0] for desc in cur.description])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

df

,station_id,infos_json,ts,tz_offset,arch_int,rev_type,temp_out,temp_out_hi,temp_out_lo,temp_in,...,wind_run,deg_days_heat,deg_days_cool,solar_energy,uv_dose,thw_index,thsw_index,wet_bulb,night_cloud_cover,iss_reception
0,122495,"{""ts"": 1632942900, ""tz_offset"": 7200, ""arch_in...",1632942900,7200,300,2,56.1,56.2,56.1,74.6,...,0.0,0.030903,0.0,0.0,NaN,55.680000,52.794106,51.865425,0.5,None
1,122495,"{""ts"": 1632943200, ""tz_offset"": 7200, ""arch_in...",1632943200,7200,300,2,56.0,56.2,56.0,73.8,...,0.0,0.031250,0.0,0.0,NaN,55.580000,52.241997,51.771530,0.5,None
2,122495,"{""ts"": 1632943500, ""tz_offset"": 7200, ""arch_in...",1632943500,7200,300,2,56.2,56.2,56.0,71.6,...,0.0,0.030556,0.0,0.0,NaN,55.780003,52.441998,51.957924,0.5,None
3,122495,"{""ts"": 1632943800, ""tz_offset"": 7200, ""arch_in...",1632943800,7200,300,2,56.3,56.4,56.2,69.3,...,0.0,0.030208,0.0,0.0,NaN,55.880000,52.541996,52.050987,0.5,None
4,122495,"{""ts"": 1632944100, ""tz_offset"": 7200, ""arch_in...",1632944100,7200,300,2,57.7,57.7,56.3,67.4,...,0.0,0.025347,0.0,0.0,0.0,56.920002,53.537514,51.984350,0.5,None


In [9]:
import psycopg2

# Connexion à la base de données
conn = psycopg2.connect(dbname=database, user=user, password=password, host=host)

# Création d'un curseur : permet d'exécuter des commandes SQL sur la base de données.
cur = conn.cursor()

# Exécution d'une requête SQL pour obtenir les informations de schéma de la table
cur.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'historiquemeteo'")

# Récupération des données dans une liste de tuples
column_info = cur.fetchall()

# Affichage du format de chaque colonne
for column in column_info:
    print("La colonne ", column[0], "à pour format :", column[1])

# Fermeture du curseur et de la connexion
cur.close()
conn.close()

La colonne  forecast_rule à pour format : double precision
La colonne  et à pour format : double precision
La colonne  abs_press à pour format : double precision
La colonne  bar_noaa à pour format : double precision
La colonne  bar à pour format : double precision
La colonne  solar_rad_avg à pour format : double precision
La colonne  dew_point_out à pour format : double precision
La colonne  dew_point_in à pour format : double precision
La colonne  solar_rad_hi à pour format : double precision
La colonne  heat_index_out à pour format : double precision
La colonne  heat_index_in à pour format : double precision
La colonne  wind_chill à pour format : double precision
La colonne  wind_run à pour format : double precision
La colonne  deg_days_heat à pour format : double precision
La colonne  deg_days_cool à pour format : double precision
La colonne  solar_energy à pour format : double precision
La colonne  uv_dose à pour format : double precision
La colonne  thw_index à pour format : doubl